In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import glob

import pandas as pd

In [8]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\'+filename}
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Total gain percentage':
                    d['tg_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain percentage':
                    d['adg_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy percentage':
                    d['bnkr_%'] = float(value.replace('%', '').strip())
                if key == 'Long':
                    d['long'] = bool(value)
                if key == 'Short':
                    d['short'] = bool(value)
                if key == 'Lowest equity/balance ratio':
                    d['ebr'] = float(value.replace('%', '').strip())

        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_145012/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_145012/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_145012/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_145012/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,tg_%,adg_%,bnkr_%,ebr,long,short
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,-100.000,-100.00000,5.983,0.909573,True,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,-100.000,-100.00000,5.949,0.114573,True,True
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,6.415,0.03490,100.000,0.838780,NaN,True
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,-100.000,-100.00000,5.941,0.952475,True,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,-100.000,-100.00000,5.999,0.242079,True,True
...,...,...,...,...,...,...,...,...,...
365,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,6.165,0.03400,100.000,0.847562,True,True
366,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,1.731,0.00975,100.000,0.841806,NaN,True
367,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,5.246,0.02990,100.000,0.972171,True,NaN
368,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,6.165,0.03400,100.000,0.847562,True,True


Now filter the dataframe

In [12]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

#allowed_symbols = ['ETHUSDT']

df = big_data[(big_data.s.isin(allowed_symbols)) &
              (big_data['bnkr_%'] > 95.0) &
              #(big_data['tg_%'] > 10) &
              (big_data['v'] == '5.3') & (big_data['long'] == True) & (big_data['short'] == True) & (big_data['ebr'] > .9)]
df

,file,v,s,tg_%,adg_%,bnkr_%,ebr,long,short
19,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.3,ALGOUSDT,9.896,0.05290,100.0,0.930943,True,True
22,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.3,ALGOUSDT,9.896,0.05290,100.0,0.930943,True,True
25,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.3,ALGOUSDT,10.540,0.05530,100.0,0.930943,True,True
28,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.3,ALICEUSDT,9.143,0.04910,100.0,0.938847,True,True
31,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.3,ALICEUSDT,9.143,0.04910,100.0,0.938847,True,True
43,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.3,ATOMUSDT,5.513,0.03020,100.0,0.936008,True,True
46,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.3,ATOMUSDT,5.513,0.03020,100.0,0.936008,True,True
49,C:\AgodaGit\passivbot\backtests/binance\BCHUSD...,5.3,BCHUSDT,3.597,0.01990,100.0,0.955068,True,True
52,C:\AgodaGit\passivbot\backtests/binance\BCHUSD...,5.3,BCHUSDT,3.597,0.01990,100.0,0.955068,True,True
97,C:\AgodaGit\passivbot\backtests/binance\COTIUS...,5.3,COTIUSDT,2.429,0.01340,100.0,0.902876,True,True


In [13]:
df.drop_duplicates(subset=['s', 'tg_%', 'bnkr_%', 'ebr', 'long', 'short'], keep='first' ,inplace=True)
df

C:\Users\skhan\AppData\Local\Temp/ipykernel_145012/2147407933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tg_%', 'bnkr_%', 'ebr', 'long', 'short'], keep='first' ,inplace=True)


,file,v,s,tg_%,adg_%,bnkr_%,ebr,long,short
19,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.3,ALGOUSDT,9.896,0.05290,100.0,0.930943,True,True
25,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.3,ALGOUSDT,10.540,0.05530,100.0,0.930943,True,True
28,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.3,ALICEUSDT,9.143,0.04910,100.0,0.938847,True,True
43,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.3,ATOMUSDT,5.513,0.03020,100.0,0.936008,True,True
49,C:\AgodaGit\passivbot\backtests/binance\BCHUSD...,5.3,BCHUSDT,3.597,0.01990,100.0,0.955068,True,True
97,C:\AgodaGit\passivbot\backtests/binance\COTIUS...,5.3,COTIUSDT,2.429,0.01340,100.0,0.902876,True,True
103,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.3,CRVUSDT,17.250,0.09070,100.0,0.932747,True,True
109,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.3,CRVUSDT,2.005,0.01120,100.0,0.902676,True,True
112,C:\AgodaGit\passivbot\backtests/binance\DOGEUS...,5.3,DOGEUSDT,6.115,0.03300,100.0,0.948894,True,True
148,C:\AgodaGit\passivbot\backtests/binance\EOSUSD...,5.3,EOSUSDT,6.973,0.03850,100.0,0.960852,True,True
